# RBI-QA Chatbot: Using Langchain | PineCone | Redis | OpenAi

In [61]:
import os
from dotenv import load_dotenv

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

load_dotenv()
# This is needed to enable LangSmith logging/tracing for chains, prompts, and LLM calls.
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

# Setting this to 'true' will start recording all your chain/agent/LLM activity for debugging and evaluation.
os.environ['LANGCHAIN_TRACING_V2'] = 'true'

# This helps organize and filter traces inside the LangSmith dashboard based on this project.
os.environ['Langchain_Project'] = os.getenv('LANGCHAIN_PROJECT')

### Utilize OenAi embeddings to convert text chunks into vectors

In [52]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

### Load and Split documents into Chunks

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader('rbi-annual-report-2023-2024.pdf')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 1000,
    chunk_overlap= 200,
    separators=["\n\n", "\n", ".", " ", ""]
)

docs = text_splitter.split_documents(documents)
docs[:5]

[Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.3 (Windows)', 'creationdate': '2024-05-29T12:25:28+05:30', 'moddate': '2024-06-03T16:37:45+05:30', 'title': '', 'trapped': '/False', 'source': 'rbi-annual-report-2023-2024.pdf', 'total_pages': 310, 'page': 1, 'page_label': '2'}, page_content='Report of the Central Board of Directors on the working of the Reserve Bank of India\nfor the year ended March 31, 2024 submitted to the Central Government in terms of  \nSection 53(2) of the Reserve Bank of India Act, 1934\nRESERVE BANK OF INDIA ANNUAL REPORT\n2023-24'),
 Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.3 (Windows)', 'creationdate': '2024-05-29T12:25:28+05:30', 'moddate': '2024-06-03T16:37:45+05:30', 'title': '', 'trapped': '/False', 'source': 'rbi-annual-report-2023-2024.pdf', 'total_pages': 310, 'page': 5, 'page_label': '6'}, page_content='GOVERNOR\nShaktikanta Das\nDEPUTY GOVERNORS\nMichael Debabrata Pa

### Initialize PineCone with Hybrid Search

In [54]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
from pinecone import ServerlessSpec, Pinecone

#Initialize Pinecone client using Pinecone Api key
pc = Pinecone(api_key=PINECONE_API_KEY)
existing_indexes = pc.list_indexes()

print("Current indexes in Pinecone Db :", pc.list_indexes().names() )

index_name = "banking-hybrid-search-with-pinecone"

try:
    if index_name not in pc.list_indexes().names():
        pc.create_index(
            index_name,
            dimension=1536,
            metric="dotproduct",
            spec=ServerlessSpec(cloud="aws", region="us-east-1")
        )
        print(f"Index '{index_name}' created successfully.")
    else:
        print(f"Index `{index_name}` already exist.")        
except Exception as e:
    print(f"Error Creating index '{index_name}: {e}")


Current indexes in Pinecone Db : ['langchain-hybrid-search-with-pinecone', 'banking-hybrid-search-with-pinecone', 'langchain-hybrid-search-pinecone']
Index `banking-hybrid-search-with-pinecone` already exist.


### Fit BM25 Encoder for Sparse Retrieval

In [55]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder.default()
sentences = [doc.page_content for doc in docs]
bm25_encoder.fit(sentences)

  0%|          | 0/1183 [00:00<?, ?it/s]

### Create Hybrid Retriever : Combine Dense and sparse retrievers from PineConeHybridSearchRetriever

In [56]:
index = pc.Index(index_name)
retriever = PineconeHybridSearchRetriever(
    index=index,
    embeddings=embeddings,
    sparse_encoder=bm25_encoder,
    top_k=4,
    alpha=0.5
)

retriever.add_texts(texts=sentences)

  0%|          | 0/37 [00:00<?, ?it/s]

### Prompt Engineering with Chain of thought, to guide the model through a reasoning process

In [57]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [79]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.output_parsers import StrOutputParser

# template with memory placeholder
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an RBI expert assistant. 
     - Do NOT answer anything outside the RBI Annual Report 2023-24.
     - If information is missing or ambiguous, respond with 'The report does not specify this'.
     - If any important information explain them in the list.
     - Explain in not more than 500 words.
     """),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

parser = StrOutputParser()

# wrap your llm with prompt
chain = chat_prompt | llm | parser

In [64]:
import redis
REDIS_URL = os.getenv('REDIS_HOST')
# Create a Redis client from the URL
r = redis.Redis.from_url(REDIS_URL, decode_responses=True)
try:
    # Test the connection
    r.set('foo', 'bar')
    print(r.get('foo'))  # Expected output: 'bar'
except Exception as e:
    print("Redis connection failed:", e)

bar


In [80]:
from langchain_community.chat_message_histories import RedisChatMessageHistory

# function provides Redis-based history based on session_id
def get_redis_history(session_id: str):
    return RedisChatMessageHistory(
        session_id=session_id,
        url=REDIS_URL
    )

In [81]:
runnable = RunnableWithMessageHistory(
    chain,
    get_redis_history,
    input_message_key="input",
    history_messages_key="chat_history"
)

In [86]:
def get_chat_response(user_input: str, session_id: str) -> str:
    """
    Invokes Langchain runnables with message history with user input and session id
    
    Args:
        user_input(str): The user's query or message
        session_id(str): Unique session identifier for tracking conversation history
    
    Returns:
        str: LLM-generated response
    """
    try:
        response = runnable.invoke(
            {"input": user_input},
            config={"configurable": {"session_id": session_id}}
        )
        return response
    except Exception as e:
        return f"An error occured {str(e)}"

In [87]:
response = get_chat_response("Tell me about RBI's monetary policy", "user_123")
print(response)

The Reserve Bank of India (RBI) formulates and implements monetary policy in India to achieve the dual objectives of maintaining price stability and promoting economic growth. The RBI's monetary policy framework is guided by the Reserve Bank of India Act, 1934, and is implemented through various tools and mechanisms.

Key aspects of RBI's monetary policy include:

1. **Monetary Policy Committee (MPC)**: The MPC is responsible for determining the policy interest rate, which is the repo rate. The MPC comprises six members, with three members nominated by the Government of India and three members from the RBI, including the Governor. The MPC meets periodically to review economic and financial indicators and decide on the appropriate stance of monetary policy.

2. **Repo Rate**: The repo rate is the rate at which the RBI lends money to commercial banks against government securities. Changes in the repo rate influence borrowing costs for banks, which, in turn, affect interest rates in the e

In [88]:
response = get_chat_response("What is repo rate", "user_123")
print(response)

The repo rate, short for repurchase rate, is the rate at which the Reserve Bank of India (RBI) lends money to commercial banks against government securities. When banks need to borrow funds from the RBI to meet their short-term liquidity needs, they can do so by providing government securities as collateral. The repo rate acts as a benchmark for short-term interest rates in the economy.

Changes in the repo rate have a direct impact on the cost of borrowing for banks. When the RBI lowers the repo rate, borrowing becomes cheaper for banks, leading to lower interest rates in the economy. This, in turn, can stimulate borrowing and investment, boosting economic activity. Conversely, when the RBI raises the repo rate, borrowing becomes more expensive, leading to higher interest rates and potentially slowing down economic growth to control inflation.

The repo rate is a key tool used by the RBI to implement monetary policy and achieve its objectives of price stability and economic growth. Th

In [89]:
response = get_chat_response("Explain more on this", "user_123")
print(response)

The repo rate is a crucial tool in the Reserve Bank of India's (RBI) monetary policy toolkit, influencing the cost of borrowing and lending in the economy. Here are some additional insights on the repo rate and its significance:

1. **Transmission Mechanism**: The repo rate serves as a signal for the overall interest rate environment in the economy. Changes in the repo rate by the RBI impact the cost of funds for banks, which, in turn, affects the interest rates they charge on loans and deposits. Lowering the repo rate encourages banks to borrow more from the RBI at cheaper rates, leading to lower lending rates in the economy. This can stimulate borrowing for investments, consumption, and overall economic activity.

2. **Inflation Management**: The repo rate plays a crucial role in the RBI's inflation targeting framework. By adjusting the repo rate, the RBI aims to manage inflation within the target range. If inflation is rising above the target, the RBI may increase the repo rate to r

In [90]:
response = get_chat_response("What happens when bank gets bankrupt", "user_123")
print(response)

The RBI Annual Report 2023-24 does not specify the exact procedures or mechanisms in place for handling a situation where a bank goes bankrupt. However, in general terms, when a bank faces insolvency or bankruptcy, there are established procedures and regulatory frameworks in place to address such situations. Here are some key points to consider:

1. **Resolution Mechanisms**: In India, the Insolvency and Bankruptcy Code (IBC) provides a framework for the resolution of insolvency and bankruptcy cases, including those involving financial institutions such as banks. The IBC outlines procedures for the initiation of insolvency proceedings, appointment of resolution professionals, and the formulation of resolution plans to address the financial distress of a bank.

2. **Regulatory Oversight**: The RBI plays a crucial role in overseeing the banking sector and ensuring financial stability. In the event of a bank facing financial difficulties, the RBI may intervene through various measures su

In [91]:
response = get_chat_response("What is Large Language Model", "user_123")
print(response)

The RBI Annual Report 2023-24 does not specifically mention or discuss Large Language Models (LLMs). Therefore, I do not have information on Large Language Models from the RBI Annual Report. If you have any other questions related to the RBI or its operations, please feel free to ask.


In [92]:
response = get_chat_response("What are FOREIGN DIRECT INVESTMENT FLOWS TO INDIA", "user_123")
print(response)

The RBI Annual Report 2023-24 provides information on Foreign Direct Investment (FDI) flows to India. However, the specific details regarding FDI flows to India in the report are not available. For detailed and up-to-date information on FDI flows to India, you may refer to the latest data and reports published by the Ministry of Commerce and Industry, Department for Promotion of Industry and Internal Trade (DPIIT), or other relevant government sources.


In [93]:
response = get_chat_response("Explain APPENDIX TABLE 9:", "user_123")
print(response)

The RBI Annual Report 2023-24 does not provide specific details about Appendix Table 9. Without the exact content or context of Appendix Table 9 from the report, I am unable to provide a detailed explanation. If you can provide the specific data or information included in Appendix Table 9, I would be happy to help analyze and explain it further. Alternatively, you may refer to the RBI Annual Report itself to access the details and context of Appendix Table 9 for a comprehensive understanding.
